In [7]:
from typing import Counter
from core import *
from pydantic import ValidationError
from pprint import pprint

repo = github_client.get_repo("pandas-dev/pandas")

import difflib

stats = Counter()
for i, pr in enumerate(repo.get_pulls(state='open')):
    if i > 10:
        print("Stopping after 10 PRs")
        break

    try:
        result = review_pull_request(repo, pr)
        print(f"PR #{pr.number}: {result}")

        if result.should_update:
            stats["should_update"] += 1

            print(f"""

                  
CHANGES FOR PR #{pr.number}:

Reason:
{result.reason}

Updated README (diff):
""")
            
            diff = difflib.unified_diff(
                repo.get_readme().decoded_content.decode("utf-8").splitlines(),
                result.updated_readme.splitlines(),
            )
            for line in diff:
                print(line)

        else:
            stats["no_update"] += 1
    except ValidationError as e:
        stats["ValidationError"] += 1
        # pprint(e.json())

pprint(stats)

PR #59855: should_update=False reason='The changes in this pull request are focused on improving the documentation of specific pandas functions (pandas.eval, DataFrame.query, and DataFrame.eval). These changes do not affect the overall project description, installation instructions, main features, or other high-level information typically included in the README file. The README already provides a comprehensive overview of the pandas project and does not require updates based on these documentation improvements.' updated_readme=None
PR #59853: should_update=False reason="The pull request is focused on implementing the `sum()` reduction for the string dtype in pandas. This is an implementation detail that doesn't affect the high-level overview or usage instructions provided in the README. The changes are internal to the pandas library and don't introduce new user-facing features or changes that would require updates to the installation, basic usage, or main features sections of the READM